<a href="https://colab.research.google.com/github/AlessandraMendes02/Projeto-IA/blob/main/Projeto_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#INSTALAÇÕES
!pip install flask transformers pillow
!pip install flask-ngrok
!pip install pyngrok
!pip install Flask-CORS
!apt-get install -y ngrok

In [ ]:
#IMPORTANDO
from pyngrok import ngrok
from flask import Flask, request, jsonify
from flask_cors import CORS
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import base64
import io

In [ ]:
#CONVERTENDO BASE64 EM IMAGEM, TREINANDO O MODELO E RETORNANDO URL DE CONEXÃO
ngrok.set_auth_token("2oysyDs6AlT4zWFCIkoE03un4oA_7yde9saoty62DksqDo5k3")

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})

genereted_url = ngrok.connect(5000)
print(f" * Ngrok URL: {genereted_url}")

@app.route('/process-image', methods=['POST'])
def process_image():
    try:
        data = request.json
        image_base64 = data.get("image")

        if not image_base64:
            return jsonify({"error": "image not found"}), 400

        image_data = base64.b64decode(image_base64.split(",")[1])
        image = Image.open(io.BytesIO(image_data)).convert('RGB')

        inputs = processor(image, return_tensors="pt")
        out = model.generate(**inputs)
        description = processor.decode(out[0], skip_special_tokens=True)

        print(f"Descrição gerada: {description}")

        return jsonify({"description": description})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

app.run(port=5000)